In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName('Python Spark SQL basic example')\
        .config('spark.some.config.option', 'some-value')\
        .getOrCreate()

### Create json file using spark
# SparkContext로 객체 생성
sc = spark.sparkContext

In [4]:
# load data
data_from_file = sc.\
    textFile(
        './data/VS14MORT.txt.gz',
        4)

In [7]:
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]

    '''
        Input record schema
        schema: n-m (o) -- xxx
            n - position from
            m - position to
            o - number of characters
            xxx - description
        1. 1-19 (19) -- reserved positions
        2. 20 (1) -- resident status
        3. 21-60 (40) -- reserved positions
        4. 61-62 (2) -- education code (1989 revision)
        5. 63 (1) -- education code (2003 revision)
        6. 64 (1) -- education reporting flag
        7. 65-66 (2) -- month of death
        8. 67-68 (2) -- reserved positions
        9. 69 (1) -- sex
        10. 70 (1) -- age: 1-years, 2-months, 4-days, 5-hours, 6-minutes, 9-not stated
        11. 71-73 (3) -- number of units (years, months etc)
        12. 74 (1) -- age substitution flag (if the age reported in positions 70-74 is calculated using dates of birth and death)
        13. 75-76 (2) -- age recoded into 52 categories
        14. 77-78 (2) -- age recoded into 27 categories
        15. 79-80 (2) -- age recoded into 12 categories
        16. 81-82 (2) -- infant age recoded into 22 categories
        17. 83 (1) -- place of death
        18. 84 (1) -- marital status
        19. 85 (1) -- day of the week of death
        20. 86-101 (16) -- reserved positions
        21. 102-105 (4) -- current year
        22. 106 (1) -- injury at work
        23. 107 (1) -- manner of death
        24. 108 (1) -- manner of disposition
        25. 109 (1) -- autopsy
        26. 110-143 (34) -- reserved positions
        27. 144 (1) -- activity code
        28. 145 (1) -- place of injury
        29. 146-149 (4) -- ICD code
        30. 150-152 (3) -- 358 cause recode
        31. 153 (1) -- reserved position
        32. 154-156 (3) -- 113 cause recode
        33. 157-159 (3) -- 130 infant cause recode
        34. 160-161 (2) -- 39 cause recode
        35. 162 (1) -- reserved position
        36. 163-164 (2) -- number of entity-axis conditions
        37-56. 165-304 (140) -- list of up to 20 conditions
        57. 305-340 (36) -- reserved positions
        58. 341-342 (2) -- number of record axis conditions
        59. 343 (1) -- reserved position
        60-79. 344-443 (100) -- record axis conditions
        80. 444 (1) -- reserve position
        81. 445-446 (2) -- race
        82. 447 (1) -- bridged race flag
        83. 448 (1) -- race imputation flag
        84. 449 (1) -- race recode (3 categories)
        85. 450 (1) -- race recode (5 categories)
        86. 461-483 (33) -- reserved positions
        87. 484-486 (3) -- Hispanic origin
        88. 487 (1) -- reserved
        89. 488 (1) -- Hispanic origin/race recode
     '''

    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs
#     return record_split.split(row)

In [8]:
data_from_file_conv = data_from_file.map(extractInformation)

# 스키마

데이터프레임과 달리 RDD는 스키마리스<sup>schema-less</sup> 데이터 구조다. 그러므로 RDD를 사용할 때 데이터셋을 병렬 처리하는 것은 전혀 문제가 되지 않는다.
그래서 튜플, 사전, 리스트와 같은 거의 모든 데이터 구조를 섞을 수 있고, 이는 스파크에서 전혀 문제가 되지 않는다.<br>
데이터셋에 대해 .collect() 함수를 수행하면 파이썬에서 일반적으로 했던 방식대로 객체 내의 데이터에 접근 할 수 있다.

In [2]:
data_heterogenous = sc.parallelize([
    ('Ferrari', 'fast'),
    {'Porsche': 100000},
    ['Spain', 'visited', 4504]
]).collect()

In [3]:
data_heterogenous[1]['Porsche']

100000

## map() 트랜스포메이션

In [9]:
data_2014 = data_from_file_conv.map(lambda row: int(row[16]))

In [10]:
data_2014.take(10)

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, -99]

In [17]:
data_2014_2 = data_from_file_conv.map(
    lambda row: (row[16], int(row[16])))
data_2014_2.take(10)

[('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('-99', -99)]

## filter() 트랜스포메이션

data_from_file_conv 데이터셋에서 2014년에 사고로 몇 명이 죽었는지를 카운트

In [20]:
data_filtered = data_from_file_conv.filter(
    lambda row: row[16] == '2014' and row[21] == '0')
data_filtered.count()

22

## flatmap(...) 트랜스포메이션
map() 함수와 비슷하게 동작. 그러나 이는 리스트가 아닌 평면화된 결과를 리턴한다.

In [21]:
data_2014_flat = data_from_file_conv.flatMap(lambda row: (row[16], int(row[16]) + 1))
data_2014_flat.take(10)

['2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015]

## distinct() 트랜스포메이션
특정 칼럼에서의 중복된 값을 제거해, 고유한 값을 리스트로 리턴.

In [22]:
distinct_gender = data_from_file_conv.map(
    lambda row: row[5]).distinct()
distinct_gender.collect()

['M', 'F', '-99']

## sample() 트랜스포메이션
 sample() 트랜스포메이션
데이터셋으로부터 임의로 추출된 샘플을 리턴한다. 첫 번째 파라미터는 중복 허용 여부 명시, 두 번째 파라미터는 리턴할 데이터셋과 전체 데이터셋 간의 크기 비율을 명시, 세 번째 파라미터는 임의의 숫자를 생성하기 위한 시드

In [23]:
fraction = 0.1
data_sample = data_from_file_conv.sample(False, fraction, 666)

In [24]:
print('Original dataset: {0}, sample: {1}'.format(data_from_file_conv.count(), data_sample.count()))

Original dataset: 2631171, sample: 263247


## leftOuterJoin() 트랜스포메이션
 leftOuterJoin() 트랜스포메이션
SQL에서처럼 두 개의 RDD를 두 개의 데이터셋에서 찾은 값에 기반에 조인하고, 두 개의 RDD가 매치되는 데이터에 대해 왼쪽 RDD에 오른쪽 RDD가 추가된 결과가 리턴

In [25]:
rdd1 = sc.parallelize([('a', 1), ('b', 4), ('c', 10)])
rdd2 = sc.parallelize([('a', 4), ('a', 1), ('b', 6), ('d', 15)])
rdd3 = rdd1.leftOuterJoin(rdd2)

In [26]:
rdd3.collect()

[('b', (4, 6)), ('c', (10, None)), ('a', (1, 4)), ('a', (1, 1))]

In [27]:
rdd4 = rdd1.join(rdd2)
rdd4.collect()

[('b', (4, 6)), ('a', (1, 4)), ('a', (1, 1))]

In [29]:
rdd5 = rdd1.intersection(rdd2)
rdd5.collect()

[('a', 1)]

## repartion() 트랜스포메이션

In [31]:
rdd1 = rdd1.repartition(4)
len(rdd1.glom().collect())

4

# 액션
트랜스포메이션과는 다르게, 액션은 데이터셋에서 스케줄된 테스크를 실행한다.

## take() 함수
map() 함수처럼 유용한 함수. 이 함수는 하나의 파티션에서 가장 위에 있는 n행을 리턴하기 때문에 RDD 전체를 리턴하는 collect()보다 더 많이 쓰인다.

## reduce() 함수
특정 함수를 사용해 RDD의 개수를 줄인다. <br>
RDD의 총합을 구하기 위해 이 함수를 사용할 수 있다.

In [32]:
rdd1.map(lambda row: row[1]).reduce(lambda x, y: x + y)

15

## count() 함수
RDD의 엘리먼트 개수를 센다.

## saveAsTextFile() 함수
 saveAsTextFile() 함수
RDD를 텍스트 파일로 저장

## foreach() 함수
RDD의 각 엘리먼트에 반복적으로 적용하는 함수 map() 함수와는 달리, foreach() 함수는 정의된 함수를 하나하나 각각의 데이터에 적용한다. 이는 Pyspark에서 지원하지 않는 데이터베이스에 데이터를 저장하고 싶을 때 유용하다.